# SatDrag Database 

Simple set of code to create a database for the Satellite Drag data.

Want to read in the csv file and output an HDF5 file which will allow for faster reading as while as searching of the file before reading and writing. 

Also add a column to the data to identify storm times. 



In [1]:
#plot matplotlib figures in the notebook
%matplotlib inline

#print all output in a cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

In [21]:
drop = ['year', 'month', 'day', 'hour', 'minute', 'second']

data_file = 'D:\\data\\VL_sdrag\\combined_data_all_reduced_omni.csv'

sd_dat = pd.read_csv(data_file)
sd_dat = sd_dat.drop(columns=drop)

,Datetime,SYM/H_INDEX_nT,1-M_AE_nT,Sec,CLat,SLat,SLon,Height,STime,DipLat,...,wavelength (nm),irradiance (W/m^2/nm),uncertainty (unitless),Total mass density,year,month,day,hour,minute,second
0,2004-02-01 00:00:00,-3.0,42.0,86392.5,6.0,7.32789,-46.97031,465.480,20.8630,9.49385,...,121.45,0.005673,0.050021,2.382877e-10,2004,2,1,0,0,0
1,2004-02-01 00:01:00,-2.0,92.0,52.5,3.0,3.46651,-47.15423,466.079,20.8674,5.99292,...,121.45,0.005673,0.050021,2.383206e-10,2004,2,1,0,1,0
2,2004-02-01 00:02:00,-2.0,130.0,115.0,0.0,-0.55515,-47.34640,466.939,20.8719,2.44186,...,121.45,0.005673,0.050021,2.383504e-10,2004,2,1,0,2,0
3,2004-02-01 00:03:00,-3.0,85.0,142.5,-3.0,-2.32433,-47.43097,467.392,20.8739,0.90719,...,121.45,0.005673,0.050021,2.383803e-10,2004,2,1,0,3,0
4,2004-02-01 00:04:00,-3.0,75.0,232.5,-9.0,-8.11216,-47.70693,469.193,20.8805,-4.01442,...,121.45,0.005673,0.050021,2.384108e-10,2004,2,1,0,4,0


In [25]:
sd_dat.to_hdf('D:\\data\\VL_sdrag\\satdrag_databas.hdf5', key='sd_dat', mode='w', format='table', data_columns='Datetime', complevel=9, complib='blosc:snappy')

In [26]:
bb = pd.read_hdf('D:\\data\\VL_sdrag\\satdrag_databas.hdf5')